In [ ]:
pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.7/767.7 kB 24.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.97.1
    Uninstalling openai-1.97.1:
      Successfully uninstalled openai-1.97.1


In [ ]:
import requests
import urllib.parse
import zipfile
import io
import json
import time
import os
import xml.etree.ElementTree as ET
import pandas as pd
import openai


# 🔐 네이버 API 인증 정보 입력
NAVER_CLIENT_ID = ""
NAVER_CLIENT_SECRET = ""

os.environ["DART_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""



## 0.2ver 상세보고서 미출력 기업 -> 뉴스기사로 상세보고서 구성

In [ ]:
import requests
import zipfile
import io
import json
import time
import os
import xml.etree.ElementTree as ET
import pandas as pd
import openai

from openai import OpenAI

# 🔑 API 키 입력
DART_API_KEY = ""
NAVER_CLIENT_ID = ""
NAVER_CLIENT_SECRET = ""

client = OpenAI(api_key="")


# ✅ 1. corpCode.xml 로딩
def load_corp_list_from_dart(api_key: str) -> pd.DataFrame:
    import requests, zipfile, io
    import xml.etree.ElementTree as ET
    import pandas as pd

    url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}"
    res = requests.get(url)

    if res.status_code != 200 or b'PK' not in res.content[:2]:
        print("❌ 응답 본문 미리보기:")
        print(res.text[:500])  # 만약 text 해석 가능한 HTML 오류면 보여줌
        raise Exception("❌ DART API에서 corpCode.zip을 받지 못했습니다.")

    try:
        z = zipfile.ZipFile(io.BytesIO(res.content))
        xml_content = z.read(z.namelist()[0])
        root = ET.fromstring(xml_content)

        data = []
        for item in root.findall('list'):
            data.append({
                "corp_code": item.findtext('corp_code'),
                "corp_name": item.findtext('corp_name'),
                "stock_code": item.findtext('stock_code'),
                "modify_date": item.findtext('modify_date')
            })

        return pd.DataFrame(data)

    except zipfile.BadZipFile:
        raise Exception("❌ 압축 파일이 아니거나 손상된 파일입니다.")

# ✅ 2. 키워드 필터링
def find_corp_by_keyword(df: pd.DataFrame, keyword: str):
    return df[df["corp_name"].str.contains(keyword, case=False, na=False)]

# ✅ 3. 기업 개황 수집
def collect_company_profile(corp_code: str, api_key: str):
    url = "https://opendart.fss.or.kr/api/company.json"
    params = {"crtfc_key": api_key, "corp_code": corp_code}
    res = requests.get(url, params=params)
    if res.status_code != 200 or res.json().get("status") != "000":
        return {}
    data = res.json()
    return {
        "기업명": data.get("corp_name"),
        "대표자명": data.get("ceo_nm"),
        "사업자등록번호": data.get("bizr_no"),
        "주소": data.get("adres"),
        "홈페이지": data.get("hm_url"),
        "업종코드": data.get("induty_code"),
        "설립일자": data.get("est_dt"),
        "상장여부": "상장" if data.get("stock_code") else "비상장",
        "지주회사": data.get("hm_ownr_corp_nm")
    }

# ✅ 4. 네이버 뉴스 검색
def search_news_titles(keyword: str):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    params = {"query": keyword, "display": 5, "sort": "date"}
    res = requests.get(url, headers=headers, params=params)
    if res.status_code != 200:
        return []
    return [item["title"] for item in res.json().get("items", [])]

# ✅ 5. GPT 요약
def gpt_summary(profile: dict, investor_type: str):
    guide = {
        "안정형": "장기적 재무 안정성과 배당 관점에서 분석해주세요.",
        "공격형": "단기 고수익 또는 고성장 관점에서 분석해주세요.",
        "융합형": "수익성과 안정성 사이 균형에 대해 분석해주세요."
    }
    news_titles = search_news_titles(profile["기업명"])
    news_str = "\n".join(f"- {title}" for title in news_titles)
    prompt = f"""
다음은 한 기업의 개요입니다. 산업 특징, 주요 사업영역, 규모, R&D 투자, 리스크 등을 요약하고,
'{investor_type}' 투자자에게 적합한 전략을 제시하세요.

[기업 개요]
- 기업명: {profile.get("기업명")}
- 대표자명: {profile.get("대표자명")}
- 업종코드: {profile.get("업종코드")}
- 상장여부: {profile.get("상장여부")}
- 설립일자: {profile.get("설립일자")}
- 주소: {profile.get("주소")}
- 지주회사: {profile.get("지주회사") or '없음'}

[관련 뉴스 제목]
{news_str}

[투자자 유형: {investor_type}]
{guide[investor_type]}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 뛰어난 금융 분석 전문가입니다. 일반인 투자자들에게 해당 기업이 각자의 투자 성향에 기반하여 투자를 권장하는지 아닌지, 어떤 투자 가치가 있는지, 쉽고 일상적인 언어와 구체적인 수치 및 사례로 설명해주세요."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"❌ GPT 요약 실패: {e}"


# ✅ 6. 보고서 저장
def save_reports(profile: dict, summary: str):
    os.makedirs("summaries", exist_ok=True)
    os.makedirs("reports", exist_ok=True)
    base_filename = profile["기업명"].replace("/", "_")

    # TXT 저장
    with open(f"summaries/{base_filename}_요약.txt", "w", encoding="utf-8") as f:
        f.write(summary)

    # JSON 저장
    react_item = {
        "name": profile.get("기업명"),
        "src": "https://res.cloudinary.com/zuzu-homepage/image/upload/f_auto,q_auto/v1739446680/p4rn9qxymke0lda94pzk.jpg",
        "itemList": [
            {"type": "text", "data": [summary]},
            {"type": "text", "data": [profile.get("상장여부")]},
            {"type": "text", "data": ["정보 없음"]},
            {"type": "text", "data": ["정보 없음"]},
            {"type": "tag", "data": ["산업", "기술", "서비스"]},
            {"type": "tag", "data": ["성장성", "시장지배력"]}
        ]
    }
    with open(f"summaries/react_items_{base_filename}.json", "w", encoding="utf-8") as f:
        json.dump(react_item, f, ensure_ascii=False, indent=2)

    # MD 저장
    with open(f"reports/{base_filename}_상세보고서.md", "w", encoding="utf-8") as f:
        f.write(f"# 📄 {profile['기업명']} 상세 보고서\n\n")
        f.write("## 🏢 기업 개요\n")
        for key, value in profile.items():
            f.write(f"- **{key}**: {value or '정보 없음'}\n")
        f.write("\n## 📘 GPT 요약\n")
        f.write(summary)

# ✅ 7. 전체 실행
def run_pipeline(keyword: str):
    corp_df = load_corp_list_from_dart(DART_API_KEY)
    matches = find_corp_by_keyword(corp_df, keyword)
    if matches.empty:
        print("❌ 해당 키워드를 포함한 기업이 없습니다.")
        return
    investor_type = input("📌 투자자 유형 (안정형 / 공격형 / 융합형): ").strip()
    if investor_type not in ["안정형", "공격형", "융합형"]:
        investor_type = "융합형"
    for _, row in matches.iterrows():
        corp_code = row["corp_code"]
        corp_name = row["corp_name"]
        print(f"\n🔍 {corp_name} (코드: {corp_code})")
        profile = collect_company_profile(corp_code, DART_API_KEY)
        if not profile:
            print("❌ 기업 개요 실패")
            continue
        summary = gpt_summary(profile, investor_type)
        print(f"\n📘 GPT 요약:\n{summary}\n")
        save_reports(profile, summary)
        time.sleep(1.2)

# ✅ 실행
if __name__ == "__main__":
    keyword = input("🔍 검색할 기업 키워드를 입력하세요: ")
    run_pipeline(keyword)



🔍 농협축산유통 (코드: 00187637)

📘 GPT 요약:
먼저, 주식회사 농협축산유통이란 회사에 대해 이야기해보겠습니다. 이 회사는 농산물과 축산물의 유통을 주요 사업으로 하고 있습니다. 이런 회사는 일반적으로 안정적인 수익을 창출하는 경향이 있습니다. 왜냐하면 농산물과 축산물은 우리가 일상에서 필수적으로 소비하는 상품이기 때문입니다. 

그럼 이제 '안정형' 투자자에게 이 회사가 얼마나 적합한지 살펴보겠습니다. 안정형 투자자는 위험을 최소화하고 장기적으로 안정적인 수익을 추구하는 투자자를 말합니다. 이런 투자자에게는 주식회사 농협축산유통이 좋은 투자 대상이 될 수 있습니다. 

먼저, 이 회사는 비상장 회사이기 때문에 주가 변동에 따른 위험이 적습니다. 또한, 농산물과 축산물은 경기에 상관없이 지속적으로 수요가 발생하는 상품이므로, 회사의 재무 상태가 안정적일 가능성이 높습니다. 

다만, 비상장 회사인 만큼 투자에 있어서는 주의가 필요합니다. 투자 전에는 반드시 회사의 재무 상태를 철저히 분석하고, 이에 대한 정보를 충분히 수집해야 합니다. 또한, 이 회사가 속한 산업의 전반적인 동향과 미래 전망도 고려해야 합니다. 

결론적으로, 주식회사 농협축산유통은 안정적인 수익을 추구하는 '안정형' 투자자에게 적합한 투자 대상이 될 수 있습니다. 그러나 투자 전에는 반드시 회사의 재무 상태와 산업 동향을 철저히 분석해야 합니다.


🔍 농협중앙회제삼차유동화전문유한회사 (코드: 00351764)

📘 GPT 요약:
먼저, 농협중앙회제삼차유동화전문 유한회사는 비상장 기업으로서, 특정한 업종에 속하지 않는 다양한 업무를 수행하는 기업입니다. 이러한 유형의 기업은 일반적으로 자금 조달을 위해 유동화증권을 발행하는 것이 특징입니다. 

관련 뉴스를 보면, 현대산업이 아이파크몰에 대해 2400억원의 채무보증을 제공했다는 내용이 있습니다. 이는 농협중앙회제삼차유동화전문 유한회사가 현대산업의 채무를 보증하고 있음을 의미합니다. 이는 일종의 안정적인 수익원이 될 수 있습니다

## 0.1ver 뉴스기사 호출

In [ ]:



# ✅ 1. corpCode.xml 로딩
def load_corp_list_from_dart(api_key: str):
    url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}"
    res = requests.get(url)
    if res.status_code != 200 or 'application/zip' not in res.headers.get('Content-Type', ''):
        raise Exception("❌ DART API에서 corpCode.zip을 받지 못했습니다.")
    z = zipfile.ZipFile(io.BytesIO(res.content))
    xml_content = z.read(z.namelist()[0])
    root = ET.fromstring(xml_content)
    data = []
    for item in root.findall('list'):
        data.append({
            "corp_code": item.findtext('corp_code'),
            "corp_name": item.findtext('corp_name'),
            "stock_code": item.findtext('stock_code'),
            "modify_date": item.findtext('modify_date')
        })
    return pd.DataFrame(data)

# ✅ 2. 키워드 포함 기업 필터링
def find_corp_by_keyword(df: pd.DataFrame, keyword: str):
    return df[df["corp_name"].str.contains(keyword, case=False, na=False)]

# ✅ 3. 기업 개요 수집
def collect_company_profile(corp_code: str, api_key: str):
    url = "https://opendart.fss.or.kr/api/company.json"
    params = {"crtfc_key": api_key, "corp_code": corp_code}
    res = requests.get(url, params=params)
    if res.status_code != 200 or res.json().get("status") != "000":
        return {}
    data = res.json()
    return {
        "기업명": data.get("corp_name"),
        "대표자명": data.get("ceo_nm"),
        "사업자등록번호": data.get("bizr_no"),
        "주소": data.get("adres"),
        "홈페이지": data.get("hm_url"),
        "업종코드": data.get("induty_code"),
        "설립일자": data.get("est_dt"),
        "상장여부": "상장" if data.get("stock_code") else "비상장",
        "지주회사": data.get("hm_ownr_corp_nm")
    }

# ✅ 4. 네이버 뉴스 기사 검색
def fetch_news_articles(query: str, display=5):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    params = {
        "query": query,
        "display": display,
        "start": 1,
        "sort": "date"
    }
    res = requests.get(url, headers=headers, params=params)
    if res.status_code != 200:
        return []
    return [item["title"] + "\n" + item["description"] for item in res.json().get("items", [])]

# ✅ 5. GPT 요약
def gpt_summary(text: str, investor_type: str = "융합형"):
    prompt = f"""
당신은 투자 전문가입니다. 아래 기업 관련 뉴스 기사들을 기반으로 기업의 산업적 특징, 성장성, 위험요소를 요약해주세요.
또한 '{investor_type}' 투자자에게 적합한 투자 전략을 제시해주세요.

[뉴스 기사 요약]
{text}
"""
    try:
        res = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5
        )
        return res.choices[0].message.content.strip()
    except Exception as e:
        return f"❌ GPT 요약 실패: {e}"

# ✅ 저장
def save_results(name: str, summary: str):
    os.makedirs("summaries", exist_ok=True)
    with open(f"summaries/{name}_news_summary.txt", "w", encoding="utf-8") as f:
        f.write(summary)
    with open(f"summaries/{name}_summary.md", "w", encoding="utf-8") as f:
        f.write(f"# 📄 {name} 뉴스 기반 요약\n\n{summary}")
    print(f"💾 저장 완료 ➜ summaries/{name}_summary.md")

# ✅ 실행
def run_news_fallback(keyword: str, investor_type: str):
    print(f"\n📡 뉴스 기반 요약 전환 중: '{keyword}'")
    articles = fetch_news_articles(keyword)
    if not articles:
        print("❌ 뉴스 검색 결과가 없습니다.")
        return
    combined = "\n\n".join(articles)
    summary = gpt_summary(combined, investor_type)
    print(f"\n📘 GPT 요약 결과:\n{summary}")
    save_results(keyword, summary)

# ✅ 최종 실행 함수
def run_pipeline(keyword: str):
    investor_type = input("📌 투자자 유형 (안정형 / 공격형 / 융합형): ").strip()
    if investor_type not in ["안정형", "공격형", "융합형"]:
        investor_type = "융합형"
    try:
        corp_df = load_corp_list_from_dart(DART_API_KEY)
        matches = find_corp_by_keyword(corp_df, keyword)
        if matches.empty:
            raise Exception("❌ 기업 없음")
        corp_code = matches.iloc[0]["corp_code"]
        profile = collect_company_profile(corp_code, DART_API_KEY)
        if not profile:
            raise Exception("❌ 개요 없음")
        summary = gpt_summary(json.dumps(profile, ensure_ascii=False), investor_type)
        save_results(keyword, summary)
    except Exception as e:
        print(str(e))
        run_news_fallback(keyword, investor_type)

# ✅ 실행
if __name__ == "__main__":
    keyword = input("🔍 검색할 기업 키워드를 입력하세요: ")
    run_pipeline(keyword)
# ✅ 주의사항
print("API 호출은 하루 1000건으로 제한됩니다.")